# Validate Qualitatively

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

from src.config import BASE_LAT_START, BASE_LAT_END, BASE_LON_START, BASE_LON_END
from data_transformer import extract_stations_from_nc
from data_provider import get_station_indices_map

In [ ]:
pred = np.load('predictions/full_pred_plain.npy')
# Convert to hecto pascal
pred[...,1] = pred[...,1] /100
print(pred.shape)

ground_truth = xr.load_dataset("data_sets/ground_truth.nc")
station_indx_map = get_station_indices_map()
gt_stations = extract_stations_from_nc(ground_truth, station_indx_map)  # Is scaled.



DATES = pd.date_range('1807-01-01', freq='D', periods=365).values
DATES = list(map(lambda d: str(d).split('T')[0], DATES))
def date_to_id(date):
    return DATES.index(date)

## Get Average Hottest and Coldest Day according to GT Observations

In [ ]:
temp_df = pd.DataFrame(gt_stations)
temp_df = temp_df[list(filter(lambda c: "_ta" in c,  temp_df.columns))]
temp_df.head(5)

In [ ]:
hottest_id = temp_df.mean(axis=1).argmax()
coolest_id = temp_df.mean(axis=1).argmin()

hottest_date = DATES[hottest_id]
coolest_date = DATES[coolest_id]

print(f"Hottest date: {hottest_date} ({hottest_id})")
print(f"Coolest date: {coolest_date} ({coolest_id})")

## Extract anomalies for those two days

In [ ]:
# TODO: based on extract_anomalies(...)
import statsmodels.api as sm

def extract_ta_cell_anomaly(y_obs, n_doy=365):
    """
    y_obs: one cell over time.
    """
    x_ = np.linspace(0, n_doy - 1, n_doy)  # doy
    x1 = np.sin((2 * np.pi * x_) / n_doy)
    x2 = np.cos((2 * np.pi * x_) / n_doy)
    x3 = np.sin((4 * np.pi * x_) / n_doy)
    x4 = np.cos((4 * np.pi * x_) / n_doy)

    x = pd.DataFrame(data={'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4})
    x_sm = sm.add_constant(x)
    model = sm.OLS(y_obs, x_sm).fit()
    c0, c1, c2, c3, c4 = model.params
    saisonal_component = c0 + c1 * x_sm.x1 + c2 * x_sm.x2 + c3 * x_sm.x3 + c4 * x_sm.x4
    desaisonalized = y_obs - saisonal_component
    # Insert nans again:
   
    return desaisonalized.to_numpy()


# FOR TEMPERATURE
# For each cell, remove seasonality.
ta_anomaly = pred[..., 0].copy()
for row in range(ta_anomaly.shape[1]):
    for column in range(ta_anomaly.shape[2]):
        ta_anomaly[:, row, column] = extract_ta_cell_anomaly(ta_anomaly[:, row, column])



# FOR PRESSURE
# Remove long-term average for each day of the year
inference_ymean = np.load("../data_sets/ymean_sets/inference_ymean.npy")
# Convert to hecto pascal
inference_ymean[...,1] = inference_ymean[...,1] /100
slp_anomaly = pred[..., 1] - inference_ymean[..., 1]

anomaly = np.concatenate([np.expand_dims(ta_anomaly, axis=-1),
                          np.expand_dims(slp_anomaly, axis=-1)], 
                         axis=-1)
print(anomaly.shape)


## Display Climatology and Anomaly Fields

In [ ]:
import cartopy.crs as ccrs

def display_single_field(pred, anomaly, hot_id, cold_id, hot_date, cold_date, savefig=None):
    fig, axes = plt.subplots(2, 2, figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
    
    fig.subplots_adjust(right=0.95)
    # Pred
    ta_hot_pred = pred[hot_id, :, :, 0]
    slp_hot_pred = pred[hot_id, :, :, 1]
    ta_cold_pred = pred[cold_id, :, :, 0]
    slp_cold_pred = pred[cold_id, :, :, 1]
    # Anomaly
    ta_hot_anomaly = anomaly[hot_id, :, :, 0]
    slp_hot_anomaly = anomaly[hot_id, :, :, 1]
    ta_cold_anomaly = anomaly[cold_id, :, :, 0]
    slp_cold_anomaly = anomaly[cold_id, :, :, 1]
    
    lon = np.linspace(BASE_LON_START, BASE_LON_END, 64)
    lat = np.linspace(BASE_LAT_START, BASE_LAT_END, 32)
    
    
    # === ROW 0 -> Climatology ===
    # --- Hot Ta ---
    axes[0,0].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[0,0].coastlines(resolution='110m', color='gray')
    axes[0,0].set_title(hot_date, fontsize=11)
    contourf_1 = axes[0,0].contourf(lon, lat, ta_hot_pred, transform=ccrs.PlateCarree(), levels=100)
    #                 tuple (left, bottom, width, height)
    sub_ax_1 = fig.add_axes([0.5, 0.53, 0.02, 0.35])
    fig.colorbar(contourf_1, cax=sub_ax_1)
    # --- Hot SLP ---
    # Add slp as contour LINES on top
    contourf_1_1 = axes[0,0].contour(lon, lat, slp_hot_pred, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[0,0].clabel(contourf_1_1, inline=True, fontsize=7)
    
    # --- Cold Ta ---
    axes[0,1].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[0,1].coastlines(resolution='110m', color='gray')
    axes[0,1].set_title(cold_date, fontsize=11)
    contourf_2 = axes[0,1].contourf(lon, lat, ta_cold_pred, transform=ccrs.PlateCarree(), levels=100)
    #                 tuple (left, bottom, width, height)
    sub_ax_2 = fig.add_axes([0.95, 0.53, 0.02, 0.35])
    fig.colorbar(contourf_2, cax=sub_ax_2)
    # --- Cold SLP ---
    # Add slp as contour LINES on top
    contourf_2_1 = axes[0,1].contour(lon, lat, slp_cold_pred, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[0,1].clabel(contourf_2_1, inline=True, fontsize=7)
    
    
    # === ROW 1 -> Anomaly ===
    # --- Hot Ta ---
    axes[1,0].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[1,0].coastlines(resolution='110m', color='gray')
    # axes[1,0].set_title(hot_date, fontsize=7)
    contourf_3 = axes[1,0].contourf(lon, lat, ta_hot_anomaly, transform=ccrs.PlateCarree(), levels=100)
    #                 tuple (left, bottom, width, height)
    sub_ax_3 = fig.add_axes([0.5, 0.11, 0.02, 0.35])
    fig.colorbar(contourf_3, cax=sub_ax_3)
    # --- Hot SLP ---
    # Add slp as contour LINES on top
    contourf_3_1 = axes[1,0].contour(lon, lat, slp_hot_anomaly, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[1,0].clabel(contourf_3_1, inline=True, fontsize=7)
    
    # --- Cold Ta ---
    axes[1,1].set_extent((BASE_LON_START, BASE_LON_END, BASE_LAT_START, BASE_LAT_END), crs=ccrs.PlateCarree())
    axes[1,1].coastlines(resolution='110m', color='gray')
    # axes[1,1].set_title(cold_date, fontsize=7)
    contourf_4 = axes[1,1].contourf(lon, lat, ta_cold_anomaly, transform=ccrs.PlateCarree(), levels=100)
    #                 tuple (left, bottom, width, height)
    sub_ax_4 = fig.add_axes([0.95, 0.11, 0.02, 0.35])
    fig.colorbar(contourf_4, cax=sub_ax_4)
    # --- Cold SLP ---
    # Add slp as contour LINES on top
    contourf_4_1 = axes[1,1].contour(lon, lat, slp_cold_anomaly, transform=ccrs.PlateCarree(), colors='black', levels=7)
    axes[1,1].clabel(contourf_4_1, inline=True, fontsize=7)
    
    fig.text(0.11, 0.56, "Absolute [°C, hPa]", fontsize=11, rotation=90)
    fig.text(0.11, 0.15, "Anomaly [°C, hPa]", fontsize=11, rotation=90)

    
    if savefig:
        plt.savefig("figures/qualitative_single_fields.png", 
            bbox_inches='tight',
            pad_inches=0.1,
            dpi=300,
            )
    
    
display_single_field(pred, anomaly, hottest_id, coolest_id, hottest_date, coolest_date, savefig=True)
    
    